In [3]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from src.databases import Postgresql

# Criando lista de skus e urls

In [4]:
#Sku
lst_sku=[]
for i in range(1,90):
    PAGE = f'https://www.wine.com.br/vinhos/cVINHOS-p{i}.html'
    if i % 10 == 0:
        print(f'Raspando pagina {i}')

    resultado = requests.get(PAGE)
    bs = BeautifulSoup(resultado.content, 'html.parser')
    html_data = bs.find_all('div', {'class': 'ProductDisplay-name'})
    sku = np.array([elem.a['data-product-sku'] for elem in html_data])
    
    for elem in html_data:
        lst_sku.append(elem.a['data-product-sku'])

Raspando pagina 10
Raspando pagina 20
Raspando pagina 30
Raspando pagina 40
Raspando pagina 50
Raspando pagina 60
Raspando pagina 70
Raspando pagina 80


In [5]:
lista = []
lista_vinhos = []

for i in range(1,90):

    r = requests.get(f'https://www.wine.com.br/browse.ep?cID=100851&exibirEsgotados=false&pn={i}&listagem=horizontal&sorter=featuredProducts-desc&filters=cVINHOS')
    soup = BeautifulSoup(r.content,'html.parser')
    container = soup.find_all('div',{'class':'ProductDisplay-name'})
    for col in container: # contém todos os vinhos
        for art in col.find_all('a',href=True): #vinhos em cada coluna
            lista = 'https://www.wine.com.br{}'.format(art['href'])
            lista_vinhos.append(lista)

In [6]:
lista_vinhos[0:3]

['https://www.wine.com.br/vinhos/maison-le-star-rose-on-ice/prod18479.html',
 'https://www.wine.com.br/vinhos/champagne-montaudon-brut/prod3170.html',
 'https://www.wine.com.br/vinhos/ropiteau-pinot-noir-2017/prod22807.html']

# Extraindo Variáveis

In [7]:
contagem = []
for i, j in enumerate(lista_vinhos):
    prods_url = j
    resultado = requests.get(prods_url)
    bs = BeautifulSoup(resultado.content, 'html.parser')
    html_data_rating = bs.find_all('a',{'class': 'Rating-count'})
    
    for elem in html_data_rating:
        if elem.attrs['href'] != "#Comments":
            contagem.append({'sku':lst_sku[i],'contagem':elem.string.strip('()')})
#price = pd.DataFrame(harmonizacao)

In [8]:
contagem[0:3]

[{'sku': '18479', 'contagem': '40'},
 {'sku': '3170', 'contagem': '193'},
 {'sku': '21768', 'contagem': '14'}]

In [10]:
#Rating

driver = webdriver.Firefox(executable_path='/usr/local/bin/geckodriver')
dados_rating=[]      
   

for i in range(1,89):
    PAGE = f'https://www.wine.com.br/vinhos/cVINHOS-p{i}.html'


    driver.get(PAGE)  
    resultado_rating = driver.find_elements_by_xpath("//*[@class='evaluation']")
    resultado_sku = driver.find_elements_by_xpath(".//*[@class='ProductDisplay-content']//*[@class='js-productClick']")
    
    for result_rating, result_sku in zip(resultado_rating,resultado_sku):
        dados_rating.append({'rating':result_rating.text,'sku':result_sku.get_attribute('data-product-sku')})


driver.quit()
#df = pd.DataFrame(dados_rating)
#df.to_csv('csv/rating.csv')

In [11]:
dados_rating[0:10]

[{'rating': '3.5', 'sku': '18479'},
 {'rating': '4.5', 'sku': '3170'},
 {'rating': '4.5', 'sku': '22807'},
 {'rating': '4.5', 'sku': '22698'},
 {'rating': '4.0', 'sku': '21768'},
 {'rating': '4.0', 'sku': '22352'},
 {'rating': '5.0', 'sku': '11420'},
 {'rating': '5.0', 'sku': '22658'},
 {'rating': '4.5', 'sku': '22710'},
 {'rating': '3.0', 'sku': '23933'}]

In [13]:
#Awards
driver = webdriver.Firefox(executable_path='/usr/local/bin/geckodriver')
dados_awards=[]      
   

for i in range(1,90):
    PAGE = f'https://www.wine.com.br/vinhos/cVINHOS-p{i}.html'

    driver.get(PAGE)  
    resultado_awards = driver.find_elements_by_xpath("//*[@class='ProductDisplay-new-awards col-xs-6']")
    resultado_sku = driver.find_elements_by_xpath(".//*[@class='ProductDisplay-content']//*[@class='js-productClick']")
    for result_awards, result_sku in zip(resultado_awards,resultado_sku):
        dados_awards.append({'Awards':result_awards.text, 'sku':result_sku.get_attribute('data-product-sku')})

driver.quit()
#df = pd.DataFrame(dados_awards, columns = ['Awards'])
#df.to_csv('csv/awards.csv')

In [48]:
dados_awards[0:3]

[{'Awards': '', 'sku': '3170'},
 {'Awards': '', 'sku': '22842'},
 {'Awards': '', 'sku': '22820'}]

In [14]:
ratingCount = pd.DataFrame(contagem).set_index('sku')
rating = pd.DataFrame(dados_rating).set_index('sku')
awards = pd.DataFrame(dados_awards).set_index('sku')

In [51]:
df = ratingCount.join([rating, awards], how = 'left')
df = df.reset_index()
df.head()

,sku,contagem,rating,Awards
0,18479,40,3.5,
1,3170,193,4.5,
2,21768,14,4.0,2
3,22352,15,4.0,
4,11420,27,5.0,


In [50]:
query = """ CREATE TABLE ratingv1 (sku          TEXT,
                                 contagem     INT,
                                 rating       REAL,
                                 Awards       TEXT) """



bd = Postgresql()
bd.create_table(query=query,user='postgres' , password='bruno22#' , host= 'localhost', port= '5432', database = 'brunods')
bd.insert_data(df = df, tabela = ' ratingv1',user='postgres' , password='bruno22#' , host= 'localhost', port= '5432', database = 'brunods')

Tabela criada no postgresql com sucesso 
Conexão com Postgresql fechada
